In [1]:
## import necessary library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

## Read the dataset, convert it into dataframe
dataframe = pd.read_csv('news.csv')
dataframe.head()

## Split data into X & Y
x = dataframe['text']
y = dataframe['label']

print(x)
print(y)

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object
0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object


In [2]:

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

## Now we need to fit the TFIDF Vectorizer.
# max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
# max_df = 25 means "ignore terms that appear in more than 25 documents".

tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

## Now let's fit the Machine Learning Model
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

## Now let's check model accuracy. Let's fit model on the test data.

y_pred = classifier.predict(tfid_x_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')
# Accuracy: 93.69%

cf = confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])
print(cf)

# [[575  40]
# [ 40 612]]

Accuracy: 93.45%
[[571  44]
 [ 39 613]]


In [3]:

## Let's create function for test the model on the real-time data.
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = classifier.predict(vectorized_input_data)
    print(prediction)

fake_news_det("""Go to Article President Barack Obama has been 
campaigning hard for the woman who is supposedly going to extend his legacy 
four more years. The only problem with stumping for Hillary Clinton, however, 
is sheâ€™s not exactly a candidate easy to get too enthused about.  """)
# ['FAKE']

# Let's save the model into pickle file so that we can use it in flask app.
import pickle
pickle.dump(classifier,open('model.pkl', 'wb'))

['FAKE']
